In [1]:
import sys; sys.path.append('..')
from osp import *

In [2]:
# !pip install scikit-learn

In [3]:
df_meta = get_corpus_metadata()

ids_phil = df_meta[df_meta['discipline'] == 'Philosophy'].index.tolist()
ids_lit = df_meta[df_meta['discipline'] == 'Literature'].index.tolist()


groups = [
    ('Philosophy', ids_phil),
    ('Literature', ids_lit),
]


In [42]:
get_pos_counts(ids_phil, incl_deprel=True, incl_pos=False)

,nsubj,cop,root,acl:relcl,obj,aux,cc,conj,obl,punct,...,vocative,advcl:relcl,nmod:unmarked,iobj,reparandum,csubj:outer,dislocated,orphan,csubj:pass,goeswith
id,,,,,,,,,,,,,,,,,,,,,
phil/10.2307/20140633__02,81,27,38,32,35,28,33,31,55,99,...,0,0,0,0,0,0,0,0,0,0
phil/10.2307/43921200__10,9,3,100,1,9,2,47,85,5,258,...,0,0,0,0,0,0,0,0,0,0
phil/10.2307/26706722__04,59,37,42,12,26,22,24,21,56,122,...,0,0,0,0,0,0,0,0,0,0
phil/10.2307/2180906__06,52,10,50,2,24,24,30,31,52,138,...,0,0,0,0,0,0,0,0,0,0
phil/10.2307/27293790__02,68,34,46,8,33,18,22,27,38,163,...,2,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
phil/10.2307/20123851__01,62,26,44,8,33,23,26,19,54,136,...,0,0,0,2,1,0,0,0,0,0
phil/10.2307/24660194__02,68,23,35,19,37,15,36,38,46,189,...,0,1,1,1,0,1,0,0,0,0
phil/10.2307/2020766__01,68,15,35,14,39,37,36,37,65,114,...,0,2,0,0,0,0,0,0,0,0


In [43]:
def classify_by_pos_counts(groups, num_runs=25, sample_size=None, incl_deprel=True, incl_pos=True, **kwargs):
    name1, ids1 = groups[0]
    name2, ids2 = groups[1]
    df_pos_grp1 = get_pos_counts(ids1, incl_deprel=incl_deprel, incl_pos=incl_pos).assign(_target=name1)
    df_pos_grp2 = get_pos_counts(ids2, incl_deprel=incl_deprel, incl_pos=incl_pos).assign(_target=name2)
    min_grp_size = min(len(ids1), len(ids2))
    if sample_size is None:
        sample_size = min_grp_size
    print(f'{name1} {len(ids1)} / {name2} {len(ids2)}')
    l_preds=[]
    l_feats=[]
    iterr = tqdm(list(range(num_runs)))
    for nrun in iterr:
        dfx1=df_pos_grp1.sample(sample_size, replace=True)
        dfx2=df_pos_grp2.sample(sample_size, replace=True)
        df_pos = pd.concat([dfx1, dfx2])
        iterr.set_description(f'{name1} {len(dfx1)} / {name2} {len(dfx2)}')
        df_preds, df_feats = classify_data(df_pos, target_col='_target', **kwargs)
        l_preds.append(df_preds.assign(run=nrun))
        l_feats.append(df_feats.assign(run=nrun))
        # df_preds['group'] = name1
        # df_preds['feat'] = df_feats['feat']
        # df_preds['feat_count'] = df_feats['feat_count']
        # df_preds['feat_pct'] = df_feats['feat_pct']
    
    odf_preds = pd.concat(l_preds) if len(l_preds) else None
    odf_feats = pd.concat(l_feats) if len(l_feats) else None
    return odf_preds, odf_feats

In [44]:
df_preds, df_feats = classify_by_pos_counts(groups, cv=10, verbose=False, incl_deprel=True, incl_pos=False, sample_size=None)

Philosophy 32277 / Literature 25343


Philosophy 25343 / Literature 25343: 100%|██████████| 25/25 [00:44<00:00,  1.79s/it]


In [45]:
df_preds.accuracy.median() * 100

np.float64(83.52405003353984)

In [46]:
df_feats.groupby('feature').weight.mean().sort_values(ascending=False)

feature
cop              29.893962
mark             29.253842
ccomp            10.118027
compound          9.914946
aux               9.777006
nmod              9.526941
expl              8.663510
amod              6.792343
acl               6.783432
aux:pass          6.256657
advmod            5.929280
root              5.492028
nsubj:pass        4.780206
acl:relcl         4.539933
fixed             3.873278
csubj             3.235429
nsubj:outer       2.928111
nsubj             2.224828
discourse         1.105388
cc:preconj        1.061458
advcl:relcl       0.545266
reparandum        0.271791
csubj:pass        0.270964
det:predet        0.125132
csubj:outer       0.015140
goeswith          0.004163
orphan           -0.103796
dislocated       -0.140539
advcl            -0.161268
dep              -0.363068
list             -0.400309
nmod:unmarked    -0.851087
vocative         -0.966124
nummod           -1.128834
obl:agent        -1.206388
obl:unmarked     -2.145973
compound:prt     -2.

In [55]:
df_preds.query('true_label != pred_label').sort_values('confidence', ascending=False)

,true_label,pred_label,test_label,confidence,correct,accuracy,support,run
id,,,,,,,,
phil/10.2307/20113565__01,Philosophy,Literature,Literature / Philosophy,0.992736,False,0.834017,50686,11
phil/10.2307/20113565__01,Philosophy,Literature,Literature / Philosophy,0.992736,False,0.834017,50686,11
phil/10.2307/20113565__01,Philosophy,Literature,Literature / Philosophy,0.992736,False,0.834017,50686,11
phil/10.2307/20113565__01,Philosophy,Literature,Literature / Philosophy,0.992625,False,0.835063,50686,2
phil/10.2307/20113565__01,Philosophy,Literature,Literature / Philosophy,0.992625,False,0.835063,50686,2
...,...,...,...,...,...,...,...,...
phil/10.2307/4544165__04,Philosophy,Literature,Literature / Philosophy,0.500024,False,0.835536,50686,13
phil/10.2307/4544165__04,Philosophy,Literature,Literature / Philosophy,0.500024,False,0.835536,50686,13
lit/10.1086/521573__03,Literature,Philosophy,Literature / Philosophy,0.500021,False,0.836168,50686,16


In [56]:
df_preds.query('true_label == pred_label').sort_values('confidence', ascending=False)


,true_label,pred_label,test_label,confidence,correct,accuracy,support,run
id,,,,,,,,
phil/10.2307/41681683__04,Philosophy,Philosophy,Literature / Philosophy,0.993816,True,0.835063,50686,2
phil/10.2307/41681683__04,Philosophy,Philosophy,Literature / Philosophy,0.993790,True,0.835063,50686,2
phil/10.2307/41681683__04,Philosophy,Philosophy,Literature / Philosophy,0.993783,True,0.835063,50686,2
phil/10.2307/41681683__04,Philosophy,Philosophy,Literature / Philosophy,0.993760,True,0.835241,50686,14
phil/10.2307/41681683__04,Philosophy,Philosophy,Literature / Philosophy,0.993724,True,0.835063,50686,2
...,...,...,...,...,...,...,...,...
phil/10.2307/20123741__04,Philosophy,Philosophy,Literature / Philosophy,0.500007,True,0.831808,50686,7
phil/10.2307/20123741__04,Philosophy,Philosophy,Literature / Philosophy,0.500007,True,0.831808,50686,7
phil/10.2307/42968793__03,Philosophy,Philosophy,Literature / Philosophy,0.500005,True,0.836661,50686,0


In [61]:
STASH_SLICES = HashStash('osp_slices_1000')
# pprint(STASH_SLICES['phil/10.2307/20129455']['1'])
# print(STASH_SLICES['phil/10.2307/41681683']['4'])

In [85]:
ycutoff = 1970
ids1 = df_meta.query(f'discipline == "Philosophy" & 1920<=year<{ycutoff}').index.tolist()
ids2 = df_meta.query(f'discipline == "Philosophy" & {ycutoff}<=year<2020').index.tolist()
sample_size = 5000

groups = [
    ('Early Philosophy', ids1),
    ('Late Philosophy', ids2),
]

df_preds, df_feats = classify_by_pos_counts(groups, cv=10, verbose=False, incl_deprel=True, incl_pos=True, sample_size=sample_size)

Early Philosophy 10037 / Late Philosophy 18948


Early Philosophy 5000 / Late Philosophy 5000: 100%|██████████| 25/25 [00:08<00:00,  2.80it/s]


In [86]:
df_preds.accuracy.median() * 100

np.float64(78.32000000000001)

In [87]:
ids1 = df_meta.query(f'discipline == "Literature" & 1920<=year<{ycutoff}').index.tolist()
ids2 = df_meta.query(f'discipline == "Literature" & {ycutoff}<=year<2020').index.tolist()

groups = [
    ('Early Literature', ids1),
    ('Late Literature', ids2),
]

df_preds2, df_feats2 = classify_by_pos_counts(groups, cv=10, verbose=False, incl_deprel=False, incl_pos=True, sample_size=sample_size)

Early Literature 9944 / Late Literature 13527


Early Literature 5000 / Late Literature 5000: 100%|██████████| 25/25 [00:08<00:00,  3.10it/s]


In [84]:
df_preds2.accuracy.median() * 100

np.float64(71.06)

In [93]:
doc = get_nlp_doc("It does not matter what matter is, is the question we must consider")
sent = doc.sentences[0]
word = sent.words[0]

In [95]:
Counter(word.pos for word in sent.words)

Counter({'AUX': 4,
         'PRON': 2,
         'VERB': 2,
         'DET': 2,
         'NOUN': 2,
         'PART': 1,
         'PUNCT': 1})

In [97]:
Counter(word.xpos for word in sent.words)

Counter({'VBZ': 3,
         'PRP': 2,
         'VB': 2,
         'NN': 2,
         'RB': 1,
         'WDT': 1,
         ',': 1,
         'DT': 1,
         'MD': 1})